# Extracting an Exoplanet Transmission Spectrum from JWST Time Series Observations


This notebook uses time series JWST NIRSpec data taken during a ground-based campain to illustrate extracting exoplanet spectra from time-series observations.  

The data are derived from the ISIM-CV3, the cryovacuum campaign of the JWST Integrated Science Instrument Module (ISIM), that took place at Goddard Space Flight Center during the winter 2015-2016 (Kimble et al. 2016). The data can be found at https://www.cosmos.esa.int/web/jwst-nirspec/test-data, and detailed and insightful report of the data by G. Giardino, S. Birkmann, P. Ferruit, B. Dorner, B. Rauscher can be found here: ftp://ftp.cosmos.esa.int/jwstlib/ReleasedCV3dataTimeSeries/CV3_TimeSeries_PRM.tgz

This NIRSpec time series dataset has had a transit light curve injected at the pixel-level, which closely mimicks a bright object time series (BOTS) observation of a transiting exoplanet. In this case, a GJ436b spectra from the 
__[Goyal et al. (2018)](https://ui.adsabs.harvard.edu/abs/2018MNRAS.474.5158G/)__ 
exoplanet grid was selected (clear atmosphere at solar metallicity).  With an actuall NIRSpec dataset, the noise properties of the detector, jitter, and the effects on extracting exoplanet spectra from time-series observations can more accuratly accuratly simulated.

Broadly the aim of this notebook is to work with these time series observations to:

 1) Extract 1D spectra from the 2D spectral images. 
    
 2) Define a time series model to fit to the wavelength dependent transit light curve.
    
 3) Fit each time series wavelength bin of the 1D spectra, measuring the desired quantity $R_{pl}(\lambda)/R_{star}$.
 
 4) Produce a measured transmission spectrum that can then be compared to models.
 
The example outputs the fit light curves for each spectral bin, along with fitting statistics.


## Load packages

This notebook uses packages (matplotlib, astropy, scipy, glob, lmfit, pickle, os, sklearn) which can all be installed in a standard fassion through pip.

Several routines to calculate limb-darkening and a transit model were extracted from ExoTiC-ISm (https://github.com/hrwakeford/ExoTiC-ISM), and slightly adapted (limb_darkening.py, transit_module.py).


In [ ]:
import numpy as np
from matplotlib.pyplot import *
from matplotlib.backends.backend_pdf import PdfPages
from astropy.utils.data import get_pkg_data_filename
from astropy.table import Table, Column, MaskedColumn
from astropy.io import fits, ascii
from scipy.interpolate import interp1d, splev, splrep
import scipy.optimize as opt
from scipy import stats
import glob
import lmfit
import limb_darkening as ld # ExoTiC-ISM Limb-Darkening (https://github.com/hrwakeford/ExoTiC-ISM)
import pickle
from os import path,mkdir
import transit_module as tm # ExoTiC-ISM Transit Model (https://github.com/hrwakeford/ExoTiC-ISM)
from sklearn.linear_model import LinearRegression
import warnings

## Setup Parameters

Parameters of the fit include directories where the data and limb darkening stellar models are held, along with properties of the planet and star. The stellar and planet values that have been entered here (modeled after GJ436) are the same as was used to model the injected transit.

In [ ]:
#---------------------------------------------------------
#    SETUP  ----------------------------------------------
#Setup directories
data_directory      ='/Users/sing/JWST_reduction_python/JWST_NIRSpec_Prism_Test_CV3_TimeSeries_PRM_2Dprocessed_injected_nl/'   #directory of DATA
limb_dark_directory ='./'                       # parent directory of ./3DGrid/ where stellar models are stored
save_directory      ='./notebookrun1/'

#Setup Detector Properties & Rednoise measurement timescale 
gain               = 1.0  # 2D spectra has already converted to counts, gain of detector is 1.0
binmeasure         = 256  # Binning technique to measure rednoise, choose bin size to evaluate sigma_r

#Setup Planet Properies
grating    = 'NIRSpecPrism'
ld_model   = '3D'              # 3D/1D

#Setup Stellar Properies for Limb-Darkening Calculation
Teff     = 4500           # Effective Temperature (K)
logg     = 4.5            # Surface Gravity
M_H      = 0.0            # Stellar Metallicity log_10[M/H]
Rstar    = 0.455          # planet radius (in units of solar radii Run)
#limb_dark = "nonlinear"  # limb darkening model

#Setup Transit parameters (can get from Nasa exoplanet archive)
t0       = 2454865.084034              # bjd time of inferior conjunction  56368.454950
per      = 2.64389803                  # orbital period (days) BJD_TDB
rp       = 0.0804                      # planet radius (in units of stellar radii)
a_Rs     = 14.54                       # semi-major axis (input a/Rstar so units of stellar radii)
inc      = 86.858  *(2*np.pi/360)      # orbital inclination (in degrees->radians)
ecc      = 0.0                         # eccentricity
omega    = 0.0  *(2*np.pi/360)         # longitude of periastron (in degrees->radians)

rho_star = (3*np.pi)/(6.67259E-8*(per*86400)**2)*(a_Rs)**3     # Stellar Density (g/cm^3) from a/Rs
# a_Rs=(rho_star*6.67259E-8*per_sec*per_sec/(3*np.pi))**(1/3)  # a/Rs from Stellar Density (g/cm^3)


In [ ]:
if path.exists(save_directory) == False: mkdir(save_directory)      #Create new directory to save outputs to if needed

## Load List of fits images

Load fits images into an array


In [ ]:
list=glob.glob(data_directory+"*.fits")
number_of_images=8192 #len(list)         #Count number of images

In [ ]:
print(list[0]) #print, first image in list

## Reading in the 2^13 fits files is slow.  

To speed things up, we create a pickle file of the for first instance the fits images are loaded.  This pickle file is loaded insead of reading the fits files if found.

# Load Fits images

The fits images are loaded, and infomation including the image date and science spectra are saved.

A default flux offset value BZERO is also taken from the header and subtracted from every science frame.


In [ ]:
if path.exists("jwst_data.pickle") == True:
    dbfile = open('jwst_data.pickle', 'rb') # for reading also binary mode is important
    jwst_data = pickle.load(dbfile)
    print('Loading JWST data from Pickle File')
    bjd            =jwst_data['bjd']
    wsdata_all     =jwst_data['wsdata_all']
    shx            =jwst_data['shx']
    shy            =jwst_data['shy']
    common_mode    =jwst_data['common_mode']
    all_spec       =jwst_data['all_spec']
    exposure_length=jwst_data['exposure_length']
    dbfile.close()    
    print('Done')
elif path.exists("jwst_data.pickle") == False:
    #---------------------------------------------------------
    #load all fits images
    #Arrays created for BJD time, and the white light curve total_counts
    index_of_images=np.arange(number_of_images) #
    bjd=np.zeros((number_of_images))
    exposure_length=np.zeros((number_of_images))
    all_spec=np.zeros((32,512,number_of_images))
    for i in index_of_images:
        img=list[i]
        print(img)
        hdul=fits.open(img)
        #hdul.info()
        bjd_image=hdul[0].header['BJD_TDB']
        BZERO=hdul[0].header['BZERO']        #flux value offset
        bjd[i]=bjd_image
        expleng=hdul[0].header['INTTIME']    #Total integration time for one MULTIACCUM (seconds)
        exposure_length[i]=expleng/86400.    #Total integration time for one MULTIACCUM (days)
        print(bjd[i])
        data = hdul[0].data
        #total counts in image
        #total_counts[i]=gain*np.sum(data[11:18,170:200]-BZERO) #total counts in 12 pix wide aperature around pixel 60 image
        all_spec[:,:,i]=gain*(data-BZERO)     #load all spectra into an array subtract flux value offset
        hdul.close()
    #Sort data
    srt=np.argsort(bjd) #index to sort
    bjd=bjd[srt]
    #total_counts=total_counts[srt]
    exposure_length=exposure_length[srt]
    all_spec[:,:,:]=all_spec[:,:,srt]

    # Get Wavelegnth of Data
    f = open('/Users/sing/JWST_reduction_python/JWST_NIRSpec_Prism_Test_CV3_TimeSeries_PRM_2Dprocessed_injected/JWST_NIRSpec_wavelength_microns.txt', 'r')
    wsdata_all = np.genfromtxt(f)
    
    print('wsdata size :',wsdata_all.shape)
    print('Data wavelength Loaded :',wsdata_all)
    print('wsdata new size :',wsdata_all.shape)
    
    #---------------------------------------------------------
    # Read in Detrending parameters
    # Mean of parameter must be 0.0 to be properly normalized
    # Idealy standard deviation of paramter = 1.0
    f = open('/Users/sing/JWST_reduction_python/JWST_NIRSpec_Prism_Test_CV3_TimeSeries_PRM_2Dprocessed_injected/JWST_NIRSpec_Xposs_Yposs_CM_detrending.txt', 'r')
    data = np.genfromtxt(f, delimiter=',')
    shx        = data[:,0]
    shy        = data[:,1]
    common_mode= data[:,2]
    
    #Store Data into a pickle file
    jwst_data={'bjd':bjd, 'wsdata_all':wsdata_all, 'shx':shx , 'shy':shy , 'common_mode':common_mode, 'all_spec':all_spec, 'exposure_length':exposure_length}
    dbfile = open('jwst_data.pickle', 'ab') # Its important to use binary mode
    pickle.dump(jwst_data,dbfile)
    dbfile.close()
    #---------------------------------------------------------

    

# Visualizing the 2D spectral data

In [ ]:
expnum=2                                           #Choose Exposure number to view

rcParams['figure.figsize'] = [10.0, 3.0]           # Figure dimensions
rcParams['figure.dpi']     = 200                   # Resolution
rcParams['savefig.dpi']    = 200
rcParams['image.aspect']   = 5                     # Aspect ratio (the CCD is quite long!!!)
cmap = matplotlib.cm.magma
cmap.set_bad('k',1.)
rcParams['image.cmap'] = 'magma'                   # Colormap.
rcParams['image.interpolation'] = None
rcParams['image.origin'] = 'lower'
rcParams['font.family'] = "monospace"
rcParams['font.monospace'] = 'DejaVu Sans Mono'

img=all_spec[:,:,expnum]
zeros=np.where(img <= 0)     #Plot on a log scale, so set zero or negitive values to a small number 
img[zeros]=1E-10
imshow(np.log10(img),vmin=0) #Plot image
xlabel('x-pixel')
ylabel('y-pixel')
axes().yaxis.set_major_locator(MultipleLocator(5))
axes().yaxis.set_minor_locator(MultipleLocator(1))
axes().xaxis.set_major_locator(MultipleLocator(50))
axes().xaxis.set_minor_locator(MultipleLocator(10))
title('2D NIRSpec Image of Exposure '+str(expnum))
colorbar(label='Log$_{10}$ Electron counts')
warnings.filterwarnings('ignore')
show()


# Extract 1D spectra from 2D array of images

Idealy, extracting 1D spectra from the 2D images would use optimal aperature extraction along a fit 
trace with routines equivalent to IRAF/apall. This functionality is not yet available in astro-py.

Several processing steps have already been applied.  The 2D spectra here have been flat field corrected, and 1/f noise has been removed from each pixel by subtracting the median count rate from the un-illuminated pixels allong each column (see __[Giardino et al.](ftp://ftp.cosmos.esa.int/jwstlib/ReleasedCV3dataTimeSeries/CV3_TimeSeries_PRM.tgz)__
for more information about 1/f noise).  Each 2D image has also been aligned in the X and Y directions, such that each pixel corresponds to the same wavelength.  As the CV3 test had no requirements for flux stability, the ~1% flux variations from the LED have also been removed.  
    
For spectral extraction, the example here simply uses a simple summed box.  The 8192 2D spectra have been
loaded into an array.  Spectra peaks at pixel Y=16, for each collumb sum over Y-axis pixels 11 to 18, which
contains most of the spectrum counts.  Wider aperature would add more counts, but also introduces more noise.

**Futher cleaning steps are not done here** 

1) Ideally, the pixels flagged as bad for various reasons should be cleaned.  

2) Cosmic rays should be identified and removed.


In [ ]:
all_spec.shape
y_lower = 11                                             # Lower extraction aperature
y_upper = 18                                             # Upper extraction aperature
all_spec_1D=np.sum(all_spec[y_lower:y_upper,:,:],axis=0) # Sum along Y-axis from pixels 11 to 18

#Plot 

rcParams['figure.figsize'] = [10.0, 3.0]           # Figure dimensions
rcParams['figure.dpi']     = 200                   # Resolution
rcParams['savefig.dpi']    = 200
rcParams['image.aspect']   = 5                     # Aspect ratio (the CCD is quite long!!!)
cmap = matplotlib.cm.magma
cmap.set_bad('k',1.)
rcParams['image.cmap'] = 'magma'                   # Colormap.
rcParams['image.interpolation'] = None
rcParams['image.origin'] = 'lower'
rcParams['font.family'] = "monospace"
rcParams['font.monospace'] = 'DejaVu Sans Mono'

img=all_spec[:,:,expnum]
zeros=np.where(img <= 0)     #Plot on a log scale, so set zero or negitive values to a small number 
img[zeros]=1E-10
imshow(np.log10(img),vmin=0) #Plot image
xlabel('x-pixel')
ylabel('y-pixel')
axes().yaxis.set_major_locator(MultipleLocator(5))
axes().yaxis.set_minor_locator(MultipleLocator(1))
axes().xaxis.set_major_locator(MultipleLocator(50))
axes().xaxis.set_minor_locator(MultipleLocator(10))
axhline(y_lower, color = 'w', ls = 'dashed')
axhline(y_upper, color = 'w', ls = 'dashed')
title('2D NIRSpec Image of Exposure '+str(expnum))
colorbar(label='Log$_{10}$ Electron counts')
import warnings
warnings.filterwarnings('ignore')
show()



# Visualizing the 1D spectral data

In [ ]:
plot(wsdata_all,all_spec_1D[:,0], linewidth=2,zorder=0)  #overplot Transit model at data
xlabel('Wavelength ($\mu$m)')
ylabel('Flux (e-)')
axes().xaxis.set_major_locator(MultipleLocator(0.5))
axes().xaxis.set_minor_locator(MultipleLocator(0.1))
annotate('H$_2$O',xy=(3.0,42000))
annotate('CO$_2$',xy=(4.2,42000))
show()

The CV3 test observed a lamp with a similar PSF as JWST will have, and has significant counts from 
about 1.5 to 4.5 $\mu$m.  

The cryogenic test chamber had CO$_2$ and H$_2$O ice buildup on the window, which can be seen as spectral absorption features in the 2D spectra.


## Calculate Orbital Phase and a seperate fine grid model used for plotting purposes



In [ ]:
#---------------------------------------------------------
#Calculate Orbital Phase
phase=(bjd-t0)/(per)  #phase in days relative to T0 ephemeris
phase=phase-np.fix(phase[number_of_images-1]) # Have current phase occur at value 0.0

t_fine = np.linspace(np.min(bjd), np.max(bjd), 1000) #times at which to calculate light curve
phase_fine=(t_fine-t0)/(per)  #phase in days relative to T0 ephemeris
phase_fine=phase_fine-np.fix(phase[number_of_images-1]) # Have current phase occur at value 0.0

b0=a_Rs * np.sqrt((np.sin(phase * 2* np.pi)) ** 2 + (np.cos(inc) * np.cos(phase * 2 * np.pi)) ** 2)
intransit=(b0-rp < 1.0E0).nonzero()  #Select indicies between first and fourth contact
outtransit=(b0-rp > 1.0E0).nonzero() #Select indicies out of transit


# Dealing with Systematic Drift On the Detector

The CV3 test assessed the stablity of the instrument by introducing a large spatial jitter and drift. This resulted in a significant X,Y movement of the spectra on the 2D detector. While this bulk shift has been removed which aligns the spectra, intra- and inter- pixel sensitivities introduce flux variations which need to be removed. The jitter from the CV3 test was more than 30 mas, which is ~4X larger than the JWST stability requiremnt. Thus, in orbit these detector effects are expected to be significantly smaller, but they will still be present and will need to be modeled and removed from time series observations.

The detector X, Y possitions were measured from cross-correlation and are saved in arrays $shx$ and $shy$. The detetctor shifts have oringinal amplitudes near 0.2 pixels, though the vectors have had inital normalization. For detrending purposes, these arrays should have a mean of 0 and standard deviation of 1.0.

A residual color-dependent trend with the LED lamp can also been seen in the data, which can be partly removed by scaling original common-mode trend, which was measured using the original white light curve.


In [ ]:
shx_tmp=shx/np.mean(shx)-1.0E0       #Set Mean around 0.0
shx_detrend=shx_tmp/np.std(shx_tmp)  #Set standard deviation to 1.0
shy_tmp=shy/np.mean(shy)-1.0E0       #Set Mean around 0.0
shy_detrend=shy_tmp/np.std(shy_tmp)  #Set standard deviation to 1.0

cm=common_mode/np.mean(common_mode)-1.0E0
cm_detrend=cm/np.std(cm)

plot(shx_detrend,label='X-possition')
plot(shy_detrend,label='Y-possition')
xlabel('Image Sequence Number')
ylabel('Relative Detector Possition')
title('Time-series Detrending Vectors')
axes().xaxis.set_major_locator(MultipleLocator(1000))
axes().xaxis.set_minor_locator(MultipleLocator(100))
axes().yaxis.set_major_locator(MultipleLocator(0.5))
axes().yaxis.set_minor_locator(MultipleLocator(0.1))
legend()
show()

## Create arrays of the vectors used for detrending.  

From Sing et al. 2019:  Systematic errors are often removed by a parameterized deterministic model, where the non-transit photometric trends are found to correlate with a number $n$ of external parameters (or optical state vectors, $X$). These parameters describe changes in the instrument or other external factors as a function of time during the observations, and are fit with a coefficient for each optical state parameter, $p_n$, to model and remove (or detrend) the photometric light curves.

When including systematic trends, the total parameterized model of the flux measurements over time, $f(t)$, can be modelled as a combination of the theoretical transit model, $T(t,\theta)$ (which depends upon the transit parameters $\theta$), the total baseline flux detected from the star, $F_0$, and the systematics error model $S(x)$ giving,

$f(t) = T(t,\theta)\times F_0 \times S(x)$.

We will use a linear model for the instrument systematic effects.

$S(x)= p_1 x + p_2 y + p_3 x^2 + p_4 y^2 + p_5 x y + p_6 cm + p_7 \phi $

$cm$ is the common_mode trend,and $\phi$ is a linear time trend which helps remove changing H$_2$O ice within the H$_2$O spectral feature.

In [ ]:
shx=shx_detrend
shy=shy_detrend
common_mode=cm_detrend

XX=np.array([shx,shy,shx**2,shy**2,shx*shy,common_mode,np.ones(number_of_images)])  #Detrending array without linear time trend
XX=np.transpose(XX)
XXX=np.array([shx,shy,shx**2,shy**2,shx*shy,common_mode,phase,np.ones(number_of_images)])  #Detrending array with with linear time trend
XXX=np.transpose(XXX)



**Linear Regression** can be used to quickly determine the parameters $p_n$ using the out-of-transit data.

Here, we take a wavelength bin of the data (pixels 170 to 200) to make a time series.  The out-of-transit points are selected and a linear regression of $S(x)$ is done to determaine the optical state parameters $p_n$

In [ ]:
wave1=170       # wavelength bin lower range
wave2=200       # wavelength bin upper range
y=np.sum(all_spec_1D[wave1:wave2,:],axis=0)    # flux over a selected wavelength bin

msize=rcParams['lines.markersize'] ** 2.           # default marker size
rcParams['figure.figsize'] = [10.0, 3.0]           # Figure dimensions
plot(wsdata_all,all_spec_1D[:,0], linewidth=2,zorder=0)  #Plot Region of wavelength bin
fill_between(wsdata_all[170:200],0,all_spec_1D[170:200,0],alpha=0.5)
xlabel('Wavelength ($\mu$m)')
ylabel('Flux (e-)')
title('1D Extracted Spectrum')
axes().xaxis.set_major_locator(MultipleLocator(0.5))
axes().xaxis.set_minor_locator(MultipleLocator(0.1))
annotate('H$_2$O',xy=(3.0,42000))
annotate('CO$_2$',xy=(4.2,42000))
show()

matplotlib.pyplot.scatter(bjd,y/np.mean(y[outtransit]),label='$f(t)$ Data',zorder=1,s=msize*0.75,linewidth=1 ,alpha=0.5, marker='+',edgecolors='blue')
xlabel('Barycentric Julian Date (days)')
ylabel('Relative Flux')
title('Time-series Transit Light Curve  $\lambda=$['+str(wsdata_all[wave1])+':'+str(wsdata_all[wave2])+'] $\mu$m')
legend()
show()

regressor = LinearRegression()
regressor.fit(XX[outtransit], y[outtransit]/np.mean(y[outtransit]))
print('Linear Regression Coefficients:')
print(regressor.coef_)

The coefficients are on the order of ~10$^{-4}$ so the trends have an amplitude on the order of 100's of ppm.

Visualize the fit

In [ ]:
yfit=regressor.predict(XX)                         # Project the fit over the whole time series
rcParams['figure.figsize'] = [10.0, 7.0]           # Figure dimensions
msize=rcParams['lines.markersize'] ** 2.           # default marker size
matplotlib.pyplot.scatter(bjd,y/np.mean(y[outtransit]),label='$f(t)$ Data',zorder=1,s=msize*0.75,linewidth=1 ,alpha=0.5, marker='+',edgecolors='blue')
plot(bjd,yfit,label='$S(x)$ Regression fit ', linewidth=2,color='orange',zorder=2,alpha=0.85)
xlabel('Barycentric Julian Date (days)')
ylabel('Relative Flux')
title('Time-series Transit Light Curve  $\lambda=$['+str(wsdata_all[wave1])+':'+str(wsdata_all[wave2])+'] $\mu$m')
axes().xaxis.set_major_locator(MultipleLocator(0.01))
axes().xaxis.set_minor_locator(MultipleLocator(0.005))
axes().yaxis.set_major_locator(MultipleLocator(0.002))
axes().yaxis.set_minor_locator(MultipleLocator(0.001))
yplot=y/np.mean(y[outtransit])
_ = ylim(yplot.min()*0.999, yplot.max()*1.001)
_ = xlim(bjd.min()-0.001, bjd.max()+0.001)
legend()
show()

# Transit Model Functions

Define a functions used by the fitting rountines. 
These which will take the transit and systematic parameters and create our full transit light curve model

$model = T(t,\theta)\times F_0 \times S(x)$

compares it to the data

$y = f(t)$

by returning the residuals 

$(y-model)/(\sigma_y)$

To calculate the transit model, here we use  __[Mandel and Agol (2002)](https://ui.adsabs.harvard.edu/abs/2002ApJ...580L.171M/abstract)__ as coded in python by H. Wakeford (__[ExoTiC-ISM](https://github.com/hrwakeford/ExoTiC-ISM)__).

A new orbit is first calculated based on the system parameters of $a/R_{star}$, the cosine of the inclination $cos(i)$, and the orbital phase $\phi$. 
The inputs are the orbit distance between the planet-star center $b$ at each phase, limb-darkening parameters ($c_1,c_2,c_3,c_4$), and the planet-to-star radius ratio $R_p/R_{star}$.

In [ ]:
#Functions to call and calculate models
def residual(p,phase,x,y,err,c1, c2, c3, c4):
    #calculate new orbit
    b0=p['a_Rs'].value * np.sqrt((np.sin(phase * 2* np.pi)) ** 2 + (p['cosinc'].value * np.cos(phase * 2 * np.pi)) ** 2)
    #Select indicies between first and fourth contact
    intransit=(b0-p['rprs'].value < 1.0E0).nonzero()
    #Make light curve model, set all values initially to 1.0
    light_curve=b0/b0
    mulimb0, mulimbf = tm.occultnl(p['rprs'].value, c1, c2, c3, c4, b0[intransit])  #Madel and Agol
    light_curve[intransit]=mulimb0
    model=(light_curve)*p['f0'].value * (p['Fslope'].value*phase + p['xsh'].value*shx + p['x2sh'].value*shx**2. + p['ysh'].value*shy + p['y2sh'].value*shy**2. + p['xysh'].value*shy*shx +  p['comm'].value*common_mode + 1.0) # transit model is baseline flux X transit model X systematics model
    chi2now=np.sum((y-model)**2/err**2)
    res=np.std((y-model)/p['f0'].value)
    print("rprs: ",p['rprs'].value,"current chi^2=",chi2now,' scatter ',res,end="\r")
    return (y-model)/err
    #return np.sum((y-model)**2/err**2)

A function is also defined to return just the transit model $T(t,\theta)$

In [ ]:
def model_fine(p):  #Make Transit model with a fine grid for plotting purposes
    b0=p['a_Rs'].value * np.sqrt((np.sin(phase_fine * 2* np.pi)) ** 2 + (p['cosinc'].value * np.cos(phase_fine * 2 * np.pi)) ** 2)
    mulimb0, mulimbf = tm.occultnl(p['rprs'].value, c1, c2, c3, c4, b0)  #Madel and Agol
    model_fine=mulimb0
    return model_fine

Now add a transit model to the Example Light curve.  Here we've pre-computed the limb darkeing coefficients. 

The transit parameters such as inclination and $a/R_{star}$ have been setup at the begining of the notebook.

In [ ]:
rl = 0.0825     # Planet-to-star Radius Ratio

b0=a_Rs * np.sqrt((np.sin(phase * 2* np.pi)) ** 2 + (np.cos(inc) * np.cos(phase * 2 * np.pi)) ** 2)
intransit=(b0-rl < 1.0E0).nonzero()  #Select indicies between first and fourth contact
c1 = 0.876820900871132      #Limb-darkeing coefficients
c2 = -0.7375522814840192
c3 = 0.5772317427624564
c4 = -0.19560435079802296
mulimb0, mulimbf = tm.occultnl(rl, c1, c2, c3, c4, b0)  #Mandel & Agol non-linear limb darkened transit model
model=mulimb0*yfit 

rcParams['figure.figsize'] = [10.0, 7.0]           # Figure dimensions
msize=rcParams['lines.markersize'] ** 2.           # default marker size
matplotlib.pyplot.scatter(bjd,y/np.mean(y[outtransit]),label='$f(t)$ Data',zorder=1,s=msize*0.75,linewidth=1 ,alpha=0.5, marker='+',edgecolors='blue')
plot(bjd,model,label='$S(x)$ Regression fit ', linewidth=2,color='orange',zorder=2,alpha=0.85)
xlabel('Barycentric Julian Date (days)')
ylabel('Relative Flux')
title('Time-series Transit Light Curve  $\lambda=$['+str(wsdata_all[wave1])+':'+str(wsdata_all[wave2])+'] $\mu$m')
axes().xaxis.set_major_locator(MultipleLocator(0.01))
axes().xaxis.set_minor_locator(MultipleLocator(0.005))
axes().yaxis.set_major_locator(MultipleLocator(0.002))
axes().yaxis.set_minor_locator(MultipleLocator(0.001))
yplot=y/np.mean(y[outtransit])
_ = ylim(yplot.min()*0.999, yplot.max()*1.001)
_ = xlim(bjd.min()-0.001, bjd.max()+0.001)
legend()
show()
err=np.sqrt(y)/np.mean(y[outtransit])
print('Chi^2 = '+str(np.sum((y/np.mean(y[outtransit])-model)**2/err**2)))

Note that the model transit depth is a little to deep compared to the data. The planet radius needs to be smaller, and the parameter $rl$ is closer to 0.08.  As an exersise you can re-run the above cell changing the planet radius to $rl$=0.0805 and compare the $\chi^2$ value to the previous default value ($\chi^2$=9265.4 at $rl$ = 0.0825).

# FIT Transit Light Curves

Now we can fit each light curve, optimzing the fit parameters with a least-squares fit. Here a Levenberg-Marquart fit is used to find a $\chi^2$ minimum and estimate uncertanties using the lmfit package (https://lmfit.github.io/lmfit-py/fitting.html).

The steps are as follows:

    1) Wavelength Bin is selected
    
    2) Limb-darkening coefficients are calculated from a stellar model for each bin. 
    
    3) An initial linear regression is performed on the out-of-transit data to start the 
    systematic fit parameters, this greatly speeds up the fit as those paramters start 
    near their global minimum.
    
    4) The fit is started, and some statistics are output during the minimization
    
    5) Once the best-fit is found, a number of statistics are displayed 
    
    6) Finally, several plots are generated which are stored as PDFs and the next bin is started.

These steps are peformed for each spectral bin.

In this example, the planet radius is set to vary in the fit along with the baseline flux and instrument systematic parameters. 

## Setup Wavelengths to fit over
The spectra must be binned in wavelength to get sufficient counts to reach ~100 ppm levels needed.
The spectra has significant counts from about pixel 100 to 400, we start at pixel $k0$ and bin the spectra by $wk$ pixels.

Several arrays are also defined.

In [ ]:
k0   = 113 #98   #100
kend = 392 #422
wk   = 15
number_of_bins = int((kend-k0)/wk)
wsd  = np.zeros((number_of_bins))
werr = np.zeros((number_of_bins))
rprs = np.zeros((number_of_bins))
rerr = np.zeros((number_of_bins))
sig_r = np.zeros((number_of_bins))
sig_w = np.zeros((number_of_bins))
beta  = np.zeros((number_of_bins))
depth = np.zeros((number_of_bins))
depth_err = np.zeros((number_of_bins))

## Loop Over Wavelength Bins Fitting Each Lightcurve
### Note this step takes considerable time to complete (~20 min, few minutes/bin)
Each wavelength bin is fit for the transit+systematics model. Various outputs including plots are saved.
Can skip to the next cells to load pre-computed results.

In [ ]:
k=k0   #wavelength to start
#--------------------------------------------------------------------------
#Loop over wavelength bins and fit for each one
for bin in range(0,number_of_bins):
    
    #---------------------------------------------------------
    # Select wavelength bin
    wave1=wsdata_all[k]
    wave2=wsdata_all[k+wk]

    #Indicies to select for wavelgth bin
    bin_wave_index = ((wsdata_all > wave1) & (wsdata_all <= wave2)).nonzero()

    #make light curve bin
    wave_bin_counts=np.sum(all_spec_1D[k+1:k+wk,:],axis=0) #Sum Wavelength pixels
    wave_bin_counts_err=np.sqrt(wave_bin_counts)           #adopt photon noise for errors
    #---------------------------------------------------------

    #---------------------------------------------------------
    # Calculate Limb Darkening

    #wsdata=wsdata_all[k:k+wk]*1E4 #limb-darkening needs angstroms
    wsdata=wsdata_all[bin_wave_index]*1E4 # Select wavelength bin values (um=> angstroms)
    #print(wsdata)
    _uLD, c1, c2, c3, c4, _cp1, _cp2, _cp3, _cp4, aLD, bLD = ld.limb_dark_fit(grating,wsdata, M_H, Teff,logg, limb_dark_directory, ld_model)

    print('\nc1 = {}'.format(c1))
    print('c2 = {}'.format(c2))
    print('c3 = {}'.format(c3))
    print('c4 = {}'.format(c4))
    print('')
    #u   = [c1,c2,c3,c4]      #limb darkening coefficients
    u   = [aLD,bLD]
    #---------------------------------------------------------

    #---------------------------------------------------------
    # Make initial model
    
    #Setup LMFIT
    x=bjd                    # X data
    y=wave_bin_counts        # Y data
    err=wave_bin_counts_err  # Y Error

    #Perform Quick Linear regression on out-of-transit data to obtain accurate starting Detector fit values
    if wave1 > 2.7 and wave1 < 3.45:
        regressor.fit(XXX[outtransit], y[outtransit]/np.mean(y[outtransit]))
    else:
        regressor.fit(XX[outtransit], y[outtransit]/np.mean(y[outtransit]))

    # create a set of Parameters for LMFIT https://lmfit.github.io/lmfit-py/parameters.html
    #class Parameter(name, value=None, vary=True, min=- inf, max=inf, expr=None, brute_step=None, user_data=None)¶
    #Set vary=0 to fix
    #Set vary=1 to fit
    p = lmfit.Parameters()  #object to store L-M fit Parameters           # Parameter Name
    p.add('cosinc'  , value=np.cos(inc)           ,vary=0)                # inclination, vary cos(inclin)
    p.add('rho_star', value=rho_star              ,vary=0)                # stellar density
    p.add('a_Rs'    , value=a_Rs                  ,vary=0)                # a/Rstar
    p.add('rprs'    , value=rp                     ,vary=1, min=0, max=1)  # planet-to-star radius ratio
    p.add('t0'      , value=t0                    ,vary=0)                # Transit T0
    p.add('f0'      , value=np.mean(y[outtransit]),vary=1, min=0)         # Baseline Flux
    p.add('ecc'     , value=ecc                   ,vary=0, min=0 , max=1) # eccentricity
    p.add('omega'   , value=omega                 ,vary=0)                # arguments of periatron
    #Turn on a linear slope in water feature to account for presumably changing H2O ice builtup on widow during cryogenic test
    if wave1 > 2.7 and wave1 < 3.45:
        p.add('Fslope', value=regressor.coef_[6]  ,vary=1)                # Orbital phase
    else:
        p.add('Fslope', value=0                   ,vary=0)                # Orbital phase
    p.add('xsh'     , value=regressor.coef_[0]    ,vary=1)                # Detector X-shift detrending
    p.add('ysh'     , value=regressor.coef_[1]    ,vary=1)                # Detector X-shift detrending
    p.add('x2sh'    , value=regressor.coef_[2]    ,vary=1)                # Detector X^2-shift detrending
    p.add('y2sh'    , value=regressor.coef_[3]    ,vary=1)                # Detector Y^2-shift detrending
    p.add('xysh'    , value=regressor.coef_[4]    ,vary=1)                # Detector X*Y detrending
    p.add('comm'    , value=regressor.coef_[5]    ,vary=1)                # Common-Mode detrending
    
    #--------------------------------------------------------------------------
    # Perform Minimization https://lmfit.github.io/lmfit-py/fitting.html
    # create Minimizer
    # mini = lmfit.Minimizer(residual, p, nan_policy='omit',fcn_args=(phase,x,y,err)
    print('')
    print('Fitting Bin',bin,' Wavelength =',np.mean(wsdata)/1E4, '  Range= [',wave1,':',wave2,']')

    #  solve with Levenberg-Marquardt using the
    result = lmfit.minimize(residual,params=p,args=(phase,x,y,err,c1, c2, c3, c4))
    #result = mini.minimize(method='emcee')

    print('')
    print('Re-Fitting Bin',bin,' Wavelength =',np.mean(wsdata)/1E4, '  Range= [',wave1,':',wave2,']')
    #--------------------------------------------------------------------------
    print("")
    print("redchi",result.redchi)
    print("chi2",result.chisqr)
    print("nfree",result.nfree)
    print("bic",result.bic)
    print("aic",result.aic)
    print("L-M FIT Variable")
    print(lmfit.fit_report(result.params))
    text_file = open(save_directory+'JWST_NIRSpec_Prism_fit_light_curve_bin'+str(bin)+'_statistics.txt', "w")
    n = text_file.write("\nredchi "+str(result.redchi))
    n = text_file.write("\nchi2   "+str(result.chisqr))
    n = text_file.write("\nnfree  "+str(result.nfree))
    n = text_file.write("\nbic    "+str(result.bic))
    n = text_file.write("\naic    "+str(result.aic))
    n = text_file.write(lmfit.fit_report(result.params))
    # file-output.py

    #Update with best-fit parameters
    p['rho_star'].value = result.params['rho_star'].value
    p['cosinc'].value   = result.params['cosinc'].value
    p['rprs'].value     = result.params['rprs'].value
    p['t0'].value       = result.params['t0'].value
    p['f0'].value       = result.params['f0'].value
    p['Fslope'].value   = result.params['Fslope'].value
    p['xsh'].value      = result.params['xsh'].value
    p['ysh'].value      = result.params['ysh'].value
    p['x2sh'].value     = result.params['x2sh'].value
    p['y2sh'].value     = result.params['y2sh'].value
    p['xysh'].value     = result.params['xysh'].value
    p['comm'].value     = result.params['comm'].value
    # Update Fit Spectra arrays
    wsd[bin]=np.mean(wsdata)/1E4
    werr[bin]=(wsdata.max()-wsdata.min())/2E4
    rprs[bin]=result.params['rprs'].value
    rerr[bin]=result.params['rprs'].stderr

    # Calculate Bestfit Model
    final_model=y-result.residual*err
    final_model_fine=model_fine(p)

    #More Stats
    resid=(y-final_model)/p['f0'].value
    residppm=1E6*(y-final_model)/p['f0'].value
    residerr=err/p['f0'].value
    sigma=np.std((y-final_model)/p['f0'].value)*1E6
    print("Residual standard deviation  (ppm) : ",1E6*np.std((y-final_model)/p['f0'].value))
    print("Photon noise                 (ppm) : ", (1/np.sqrt(p['f0'].value))*1E6     )
    print("Photon noise performance       (%) : ", (1/np.sqrt(p['f0'].value))*1E6 / (sigma) *100 )
    n = text_file.write("\nResidual standard deviation  (ppm) : "+str(1E6*np.std((y-final_model)/p['f0'].value)))
    n = text_file.write("\nPhoton noise                 (ppm) : "+str((1/np.sqrt(p['f0'].value))*1E6))
    n = text_file.write("\nPhoton noise performance       (%) : "+str((1/np.sqrt(p['f0'].value))*1E6 / (sigma) *100 ))
 
    #Measure Rednoise with Binning Technique
    sig0=np.std(resid)
    bins=number_of_images/binmeasure
    wsb, wsb_bin_edges,binnumber = stats.binned_statistic(bjd,resid, bins=bins)
    sig_binned=np.std(wsb)
    sigrednoise=np.sqrt(sig_binned**2-sig0**2/binmeasure)
    if np.isnan(sigrednoise) == True : sigrednoise=0   #if no rednoise detected, set to zero
    sigwhite   =np.sqrt(sig0**2-sigrednoise**2)
    sigrednoise=np.sqrt(sig_binned**2-sigwhite**2/binmeasure)
    if np.isnan(sigrednoise) == True : sigrednoise=0   #if no rednoise detected, set to zero
    beta[bin]=np.sqrt(sig0**2+binmeasure*sigrednoise**2)/sig0
    
    print("White noise                  (ppm) : ",1E6*sigwhite)
    print("Red noise                    (ppm) : ",1E6*sigrednoise)
    print("Transit depth measured error (ppm) : ",2E6*result.params['rprs'].value*result.params['rprs'].stderr)
    
    n = text_file.write("\nWhite noise                  (ppm) : "+str(1E6*sigwhite))
    n = text_file.write("\nRed noise                    (ppm) : "+str(1E6*sigrednoise))
    n = text_file.write("\nTransit depth measured error (ppm) : "+str(2E6*result.params['rprs'].value*result.params['rprs'].stderr))
    text_file.close()
    depth[bin]=1E6*result.params['rprs'].value**2
    depth_err[bin]=2E6*result.params['rprs'].value*result.params['rprs'].stderr

    sig_r[bin]=sigrednoise*1E6
    sig_w[bin]=sigwhite*1E6
    #--------------------------------------------------------------------------
    #---------------------------------------------------------
    #Write Fit Spectra to ascii file
    ascii_data = Table([wsd, werr, rprs, rerr,depth,depth_err,sig_w,sig_r,beta], names=['Wavelength Center (um)', 'Wavelength half-width (um)','Rp/Rs','Rp/Rs 1-sigma error','Transit Depth (ppm)','Transit Depth error','Sigma_white (ppm)','Sigma_red (ppm)','Beta Rednoise Inflation factor'])
    ascii.write(ascii_data, save_directory+'JWST_NIRSpec_Prism_fit_transmission_spectra.csv', format='csv',overwrite=True)
    #---------------------------------------------------------
    #plot transmission spectra results, keep updating during fit loop
    f = open('trans-iso_GJ-0436_0669.0_+0.0_0.56_0001_0.00_model.NIRSpec_PRISM.txt', 'r')
    data = np.genfromtxt(f, delimiter='   ')
    model_ws   = data[:,0]
    model_spec = data[:,1]
    clf()
    #
    plot(model_ws,model_spec, linewidth=2,zorder=0,color='blue',label='Injected Spectra')  #overplot Transit model at data
    fitindex = (wsd > 0).nonzero()
    errorbar(wsd[fitindex],rprs[fitindex],xerr=werr[fitindex],yerr=rerr[fitindex]*beta[fitindex], fmt='o',color='orange',zorder=5,alpha=0.4,label='Recovered Spectra with rednoise')
    errorbar(wsd[fitindex],rprs[fitindex],xerr=werr[fitindex],yerr=rerr[fitindex], fmt='o',color='orange',zorder=10,label='Recovered Spectra')
    xlabel('Wavelength ($\mu$m)')
    ylabel('Planet-to-Star Radius Ratio ($R_p/R_s$)')
    axes().yaxis.set_major_locator(MultipleLocator(0.001))
    axes().yaxis.set_minor_locator(MultipleLocator(0.005))
    axes().xaxis.set_major_locator(MultipleLocator(0.5))
    axes().xaxis.set_minor_locator(MultipleLocator(0.1))
    #_ = ylim(model_spec.min()*0.975, model_spec.max()*1.025)
    _ = xlim(0.9,5.25)
    legend()
    pp = PdfPages(save_directory+'JWST_NIRSpec_Prism_fit_transmission_spectra_bin_rprs.pdf')
    savefig(pp,format='pdf')
    pp.close()
    clf()
    #--------------------------------------------------------------------------
    #Plot data models
    msize=rcParams['lines.markersize'] ** 2. #default marker size
    matplotlib.pyplot.scatter(x,y/p['f0'].value,s=msize*0.75, linewidth=1,zorder=0,alpha=0.5, marker='+',edgecolors='blue')
    xlabel('BJD')
    ylabel('Relative Flux')
    plot(x,final_model/p['f0'].value, linewidth=1,color='orange',alpha=0.8,zorder=15)  #overplot Transit model at data
    yplot=y/p['f0'].value
    _ = ylim(yplot.min()*0.999, yplot.max()*1.001)
    _ = xlim(bjd.min()-0.001, bjd.max()+0.001)
    #show()
    pp = PdfPages(save_directory+'JWST_NIRSpec_Prism_fit_light_curve_bin'+str(bin)+'_lightcurve.pdf')
    savefig(pp,format='pdf')
    pp.close()
    clf()
    #---------------------------------------------------------
    #Plot Residuals
    msize=rcParams['lines.markersize'] ** 2. #default marker size
    matplotlib.pyplot.scatter(x,residppm, s=msize*0.75,linewidth=1 ,alpha=0.5, marker='+',edgecolors='blue',zorder=0)  #overplot Transit model at data
    wsb, wsb_bin_edges,binnumber = stats.binned_statistic(bjd,residppm, bins=256)
    xlabel('BJD')
    ylabel('Fit Residuals (ppm)')
    plot([bjd.min(),bjd.max()],[0,0],color='black',zorder=10)
    plot([bjd.min(),bjd.max()],[sigma,sigma],linestyle='--',color='black',zorder=15)
    plot([bjd.min(),bjd.max()],[-sigma,-sigma],linestyle='--',color='black',zorder=20)
    matplotlib.pyplot.scatter(wsb_bin_edges[1:],wsb, linewidth=2,alpha=0.75,facecolors='orange',edgecolors='none', marker='o',zorder=25)
    _ = ylim(residppm.min()*1.1, residppm.max()*1.1)
    _ = xlim(bjd.min()-0.001, bjd.max()+0.001)
    #show()
    pp = PdfPages(save_directory+'JWST_NIRSpec_Prism_fit_light_curve_bin'+str(bin)+'_residuals.pdf')
    savefig(pp,format='pdf')
    pp.close()
    clf()
    #--------------------------------------------------------------------------
    #plot systematic corrected light curve
    b0=p['a_Rs'].value * np.sqrt((np.sin(phase * 2* np.pi)) ** 2 + (p['cosinc'].value * np.cos(phase * 2 * np.pi)) ** 2)
    intransit=(b0-p['rprs'].value < 1.0E0).nonzero()
    light_curve=b0/b0
    mulimb0, mulimbf = tm.occultnl(p['rprs'].value, c1, c2, c3, c4, b0[intransit])  #Madel and Agol
    light_curve[intransit]=mulimb0
    #Plot data models
    msize=rcParams['lines.markersize'] ** 2. #default marker size
    matplotlib.pyplot.scatter(x,light_curve+resid,s=msize*0.75, linewidth=1,zorder=0,alpha=0.5, marker='+',edgecolors='blue')
    xlabel('BJD')
    ylabel('Relative Flux')
    plot(x,light_curve, linewidth=2,color='orange',alpha=0.8,zorder=15)  #overplot Transit model at data
    yplot=light_curve+resid
    _ = ylim(yplot.min()*0.999, yplot.max()*1.001)
    _ = xlim(bjd.min()-0.001, bjd.max()+0.001)
    #show()
    pp = PdfPages(save_directory+'JWST_NIRSpec_Prism_fit_light_curve_bin'+str(bin)+'_corrected.pdf')
    savefig(pp,format='pdf')
    pp.close()
    clf()
    #--------------------------------------------------------------------------
    k=k+wk  #step wavelength index to next bin
    
    print('** Can Now View Output PDFs in ',save_directory)


# Plot Measured Exoplanet Transmission Spectrum vs Injected 

In [ ]:
#--------------------------------------------------------------------------
#Load Injected Transmission spectra to compare with recovered value
f = open('trans-iso_GJ-0436_0669.0_+0.0_0.56_0001_0.00_model.NIRSpec_PRISM.txt', 'r')
data = np.genfromtxt(f, delimiter='   ')
model_ws   = data[:,0]
model_spec = data[:,1]

data = ascii.read(save_directory+'JWST_NIRSpec_Prism_fit_transmission_spectra.csv', format='csv')
wsd  = data['Wavelength Center (um)']
werr = data['Wavelength half-width (um)']
rprs = data['Rp/Rs']
rerr = data['Rp/Rs 1-sigma error']
beta = data['Beta Rednoise Inflation factor']

plot(model_ws,model_spec**2*1E6, linewidth=2,zorder=0,color='blue',label='Injected Spectra')  #overplot Transit model at data
errorbar(wsd,rprs**2*1E6,xerr=werr,yerr=2*rerr*rprs*1E6*beta, fmt='o',zorder=5,alpha=0.4,color='orange',label='Recovered Spectra with $\sigma_r$')
errorbar(wsd,rprs**2*1E6,xerr=werr,yerr=2*rerr*rprs*1E6, fmt='o',zorder=10,color='orange',label='Recovered Spectra')
xlabel('Wavelength ($\mu$m)')
ylabel('Transit Depth ($R_p/R_s$)$^2$ (ppm)')
axes().yaxis.set_major_locator(MultipleLocator(100))
axes().yaxis.set_minor_locator(MultipleLocator(50))
axes().xaxis.set_major_locator(MultipleLocator(0.5))
axes().xaxis.set_minor_locator(MultipleLocator(0.1))
#_ = ylim(((model_spec.min())**2*0.95*1E6), ((model_spec.max())**2*1.025*1E6))
_ = xlim(0.9,5.25)
rcParams["legend.loc"] = 'lower right' 
legend()
show()



By in large, the injected transit depths are well recovered accross the spectra, with features such as H$_2$O and CH$_4$ easily detected. There is a bit of an offset in datapoints longward of 3.5 $\mu$m that could perhaps be due to changes in CO$_2$ or H$_2$O absorption features from ice built up on the cyrogenic window during the CV3 test. These wavelengths show some increases in time correlated noise ($\sigma_r$), which has been measured here, and the errors in the plot also show the transit depths with this error included. 

The precisions from the ground-based test are encoraging, with the best measured bin (which occurs in a clean part of the spectrum with high count rates) achiveing near-photon limited transit depths measured to about 30 ppm in only 2 hours of data, and with minimial time correlated noise ($\sigma_r$).

For more robust error estimates, in practice the least-squares minimization performed here would be replaced by an MCMC routine.  In addition, the transit fit parameters (e.g. $i$, $a/R_{star}$, T$_0$) would also have to be first fit as well, as they can/will differ from literature estimates in high precision transit light curves as JWST will provide.
